In [11]:
import os
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import (
    Dense, LSTM, LSTMCell, Dropout
)
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from keras.optimizers import adam_v2, sgd_experimental

from sklearn.metrics import (
    mean_absolute_error as mae, 
    mean_squared_error as mae
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from bunch import bunchify

from django.conf import settings

In [5]:
DATASET_BASEDIR = f"{settings.BASE_DIR}/labs/datasets"

In [3]:
currency = 'usd'

In [6]:
usd = pd.read_csv(f"{DATASET_BASEDIR}/usd_datasets.csv")
usd.head()

,Date,Price,Open,High,Low
0,2014-01-01,12170.0,12170.0,12170.0,12170.0
1,2014-01-02,12160.0,12195.0,12260.0,12150.0
2,2014-01-03,12170.0,12160.0,12245.0,12160.0
3,2014-01-06,12180.0,12195.0,12245.0,12180.0
4,2014-01-07,12237.5,12197.5,12277.5,12197.5


# Data Single Fitur Configuration

Pada tahap ini data yang diproses hanya fitur data ***Price*** saja. fitur  ***Open***, ***High***, dan ***Low*** diabaikan.

> **Notes**
>
> perform the process of sharing training and test data, then the data will be normalized in standard form. 

## Data spliting (training & testing)

> **Notes**
> Pembagian data training dan testing menggunakan rasio 80% awal untuk training dan 20% terakhir sebagai testing

In [7]:
# split function

def split(data, n_train=0.8, n_test=0.2):
    train_set = data.Price.iloc[:round(n_train*len(data))].values
    test_set = data.Price.iloc[round( n_train*len(data) ):].values
    return train_set, test_set

In [8]:
dataset={}
dataset['train_set'], dataset['test_set'] = split(usd)

## Scaling data with MinMaxFunction [-1, 1]

\begin{equation}
X\_std = \frac{({X} - {X.min(axis=0)})} {(X.max(axis=0) - X.min(axis=0))}\\
\\
X\_scaled = X\_std * (max - min) + min\\
\end{equation}


In [12]:
# define scaler parameter
scaler = MinMaxScaler(feature_range=(-1, 1))

In [13]:
# data set transformation to metric shape (n_data, 1)
dataset['train_set'] = dataset['train_set'].reshape(
    len(dataset['train_set']), 1
)
dataset['test_set'] = dataset['test_set'].reshape(
    len(dataset['test_set']), 1
)

In [14]:
# data transformation
dataset['train_set_scaled'] = scaler.fit_transform(dataset['train_set'])
dataset['test_set_scaled'] = scaler.fit_transform(dataset['test_set'])

In [15]:
# convert dict to object

class dict2obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
                setattr(self, a, [dict2obj(x) if isinstance(x, dict) else x for x in b])
            else:
                setattr(self, a, dict2obj(b) if isinstance(b, dict) else b)

In [16]:
data = dict2obj(dataset)

# Sliding dataset

In [17]:
def sliding_window(data, window, step_size=1):
    X_train=[]
    y_train=[]
    
    for i in range(window, data.shape[0]):
        X_train.append(data[i-window:i])
        y_train.append(data[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    return X_train, y_train

In [18]:
data.windows={}

In [19]:
inputs = [7,8,9,10]

for window in inputs:
    data.windows[f'X_train{window}'], data.windows[f'y_train{window}'] = sliding_window(
        data=data.train_set_scaled,
        window=window
    )
    data.windows[f'X_test{window}'], data.windows[f'y_test{window}'] = sliding_window(
        data=data.test_set_scaled,
        window=window
    )

In [20]:
data.windows=dict2obj(data.windows)

In [21]:
data.windows.X_train8.shape

(1439, 8, 1)

# Proposed model design and Experiment design
| No. | Hyperparameters | Accuracy |
| :- | -: | :-: |
| 1* | Unit input/Sliding Window | [7,8,9,10] |
| 2* | Unit Neuron/Block LSTM | [1,2,3,4] |
| 3 | Unit Output/Target | 1 |
| 4 | Epochs | 100 |
| 5 | Hidden Layer (Stacked) | 3 |
| 6* | Optimizer | SGD, Adam |
| 7 | Beta Value | B1 = 0,9 & B2 = 0,99 |
| 8* | Learning Rate | 0,1; 0,01; 0,001; 0,0001 |
| 9 | Batch Size | 32 |
| 10* | Dropout | 0%, 20% |
| 11 | Activate Func | sigmoid & tanh |

# Learning Rate 
Eksperimen dilakukan dengan cara mencari learning rate paling sesuai. pengujian dilakukan dengan nilai 1, 0,1, 0,01 dan 0,001 untuk nilai Learning Rate. Parameter lain akan diatur secara konstan, yaitu sliding window sebesar 1, 2 unit neuron pada hidden layer dan epoch maksimum 1000..

In [24]:
data.lr={}
data.lr['X_train'], data.lr['y_train'] = sliding_window(
    data=data.train_set_scaled,
    window=1
)
data.lr['X_test'], data.lr['y_test'] = sliding_window(
    data=data.test_set_scaled,
    window=1
)

In [25]:
data.lr = dict2obj(data.lr)

In [26]:
n_data = data.lr.X_train.shape[0]
X_train = data.lr.X_train[:round(n_data*.8)]
y_train = data.lr.y_train[:round(n_data*.8)]
X_validation = data.lr.X_train[round(n_data*.8):]
y_validation = data.lr.y_train[round(n_data*.8):]

## LR = 1